In [2]:
from pandas import *
import time
from nebula3.common.ttypes import ErrorCode

from nebula3.gclient.net import Connection
from nebula3.gclient.net.SessionPool import SessionPool
from nebula3.Config import SessionPoolConfig
from nebula3.common import *
from FormatResp import print_resp

In [2]:
df = read_csv("data_test.csv", sep=';')  #, index_col='id события') , parse_dates=['date']).drop('id', axis=1)
df.head()

,id события,ФИО участника события 1,ФИО участника события 2
0,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
1,206,Офицеров Олег Романович,Сапожник Борис Валерьевич
2,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович
3,503,Масимова Яна Дамировна,Мингажетдинов Рамиль Семенович
4,571,Мухтарова Алена Яковлевна,Щербатенко Ольга Робертовна


In [21]:
for i in zip(df['id события'], df['ФИО участника события 1'], df['ФИО участника события 2']):
    print(i)


(189, 'Галчевская Карина Владимировна', 'Белоновская Анастасия Семеновна')
(206, 'Офицеров Олег Романович', 'Сапожник Борис Валерьевич')
(445, 'Жандарова Лариса Германовна', 'Чемодуров Дамир Русланович')
(503, 'Масимова Яна Дамировна', 'Мингажетдинов Рамиль Семенович')
(571, 'Мухтарова Алена Яковлевна', 'Щербатенко Ольга Робертовна')
(595, 'Русских Кира Константиновна', 'Федутинов Артем Артурович')
(637, 'Федонкина Кристина Маратовна', 'Маргиев Григорий Иванович')
(741, 'Чернолусская Екатерина Марселевна', 'Бухановская Лидия Радиковна')
(996, 'Оранский Владимир Артурович', 'Скотников Виктор Ильич')
(1210, 'Ардатов Андрей Анатольевич', 'Ханыгин Дмитрий Юрьевич')
(1392, 'Рифтин Кирилл Анатольевич', 'Гальчевская Полина Васильевна')
(1543, 'Балог Вадим Никитович', 'Абушаева Екатерина Юрьевна')
(1679, 'Голиборода Жанна Александровна', 'Бычек Виктор Рамилевич')
(1903, 'Слободяникова Галина Германовна', 'Васюхнов Глеб Леонидович')
(1935, 'Вахрушкова Екатерина Маратовна', 'Лифенко Марсель Конс

In [3]:
v = df['ФИО участника события 1'].unique()
print(len(v))
v

4930


array(['Галчевская Карина Владимировна', 'Офицеров Олег Романович',
       'Жандарова Лариса Германовна', ..., 'Востоков Виктор Ильдарович',
       'Огарева Людмила Ильдаровна', 'Ряполовский Георгий Петрович'],
      dtype=object)

In [4]:
v2 = df['ФИО участника события 2'].unique()
print(len(v2))
v2

4994


array(['Белоновская Анастасия Семеновна', 'Сапожник Борис Валерьевич',
       'Чемодуров Дамир Русланович', ..., 'Аликас Никита Андреевич',
       'Нагайцева Алина Степановна', 'Жилейкин Виктор Павлович'],
      dtype=object)

In [5]:
vv = list(v) + list(v2)
print(len(vv))
vv
vvv = set(vv)
print(len(vvv))
vertex = list(vvv)
vertex

9924
9899


['Типикина Карина Руслановна',
 'Чикало Тимур Юрьевич',
 'Ларьков Радик Кириллович',
 'Арндт Алексей Михаилович',
 'Бекишев Александр Артурович',
 'Осокина Диана Максимовна',
 'Ивакина Жанна Антоновна',
 'Кипарская Кристина Ивановна',
 'Сосименко Ефим Николаевич',
 'Самсонова Наталья Ивановна',
 'Соломахина Лидия Ильдаровна',
 'Сипачева Диана Маратовна',
 'Пивень Виктор Иванович',
 'Дуленкова Раиса Артемовна',
 'Хитулин Артем Александрович',
 'Можевитинов Егор Андреевич',
 'Объедкова Галина Вячеславовна',
 'Евстратиков Денис Владиславович',
 'Хандошкин Марсель Викторович',
 'Бутырев Ефим Валерьевич',
 'Острогорский Дмитрий Станиславович',
 'Таловеров Станислав Владимирович',
 'Рочагов Тимур Янович',
 'Мокрецова Алина Юрьевна',
 'Зина Инна Юрьевна',
 'Вельмукина Яна Георгиевна',
 'Мухаметзянов Антон Павлович',
 'Войтинский Никита Ильич',
 'Очиров Вадим Васильевич',
 'Кожарский Роберт Петрович',
 'Петросян Степан Андреевич',
 'Жлуктов Андрей Николаевич',
 'Зубайдуллина Елизавета Робертов

In [16]:
edge = df['id события'].unique()
print(len(edge))
edge

4985


array([   189,    206,    445, ..., 999403, 999405, 999878])

In [9]:
if __name__ == '__main__':
    ip = '127.0.0.1'
    port = 9669

    
    config = SessionPoolConfig()

    # подготовьте пространство
    conn = Connection()
    conn.open(ip, port, 1000)
    auth_result = conn.authenticate('root', 'satelite')
    
    resp = conn.execute(
        auth_result._session_id,
        'CREATE SPACE IF NOT EXISTS test(vid_type=FIXED_STRING(50))',
    )

In [10]:
# пул сеансов инициализации init session pool
session_pool = SessionPool('root', 'satelite', 'test', [(ip, port)])
session_pool.init(config)

True

In [11]:
session_pool.execute('USE test')

ResultSet(None)

In [12]:
# добавить схему add schema
resp = session_pool.execute(
            'CREATE TAG IF NOT EXISTS person(name string);'
            'CREATE EDGE communication(id int);'
        )

In [14]:
# вставить вершину insert vertex
for name in vertex:
    resp = session_pool.execute(f'INSERT VERTEX person(name) VALUES "{name}":("{name}")')

In [102]:
# вставить ребро insert edges
for iv_id, name1, name2 in zip(df['id события'], df['ФИО участника события 1'], df['ФИО участника события 2']):
    resp = session_pool.execute(f'INSERT EDGE communication(id) VALUES "{name1}"->"{name2}":({iv_id});')

In [104]:
resp = session_pool.execute('INSERT VERTEX person(name) VALUES 10000:("Гудеева Оксана Ефимовна")')

In [21]:
if __name__ == '__main__':
    ip = '127.0.0.1'
    port = 9669

    try:
        config = SessionPoolConfig()

        # подготовьте пространство
        conn = Connection()
        conn.open(ip, port, 1000)
        auth_result = conn.authenticate('root', '')
        assert auth_result.get_session_id() != 0
        resp = conn.execute(
            auth_result._session_id,
            'CREATE SPACE IF NOT EXISTS session_pool_test(vid_type=FIXED_STRING(30))',
        )
        assert resp.error_code == ErrorCode.SUCCEEDED
        # вставленные данные необходимо перевести в режим ожидания после создания схемы insert data need to sleep after create schema
        time.sleep(10)

        # пул сеансов инициализации init session pool
        session_pool = SessionPool('root', 'nebula', 'session_pool_test', [(ip, port)])
        assert session_pool.init(config)

        # добавить схему add schema
        resp = session_pool.execute(
            'CREATE TAG IF NOT EXISTS person(name string, age int);'
            'CREATE EDGE like (likeness double);'
        )
        # вставить вершину insert vertex
        resp = session_pool.execute(
            'INSERT VERTEX person(name, age) VALUES "Bob":("Bob", 10), "Lily":("Lily", 9)'
        )
        assert resp.is_succeeded(), resp.error_msg()

        # вставить ребро insert edges
        resp = session_pool.execute(
            'INSERT EDGE like(likeness) VALUES "Bob"->"Lily":(80.0);'
        )
        assert resp.is_succeeded(), resp.error_msg()

        # resp = session_pool.execute('FETCH PROP ON person "Bob" YIELD vertex as node')
        # assert resp.is_succeeded(), resp.error_msg()
        # print_resp(resp)

        # resp = session_pool.execute('FETCH PROP ON like "Bob"->"Lily" YIELD edge as e')
        # assert resp.is_succeeded(), resp.error_msg()
        # print_resp(resp)

        # drop space
        conn.execute(
            auth_result._session_id,
            'DROP SPACE session_pool_test',
        )

        print("Example finished")

    except Exception as x:
        import traceback

        print(traceback.format_exc())
        exit(1)

Traceback (most recent call last):
  File "/tmp/ipykernel_2821/1834141471.py", line 17, in <module>
    assert resp.error_code == ErrorCode.SUCCEEDED
AssertionError

